# Authentic vs Fake Bank Note Classification

## Summary

The goal of this project is to determine whether a number of random bank notes can be accurately classified as authentic or fake. Through the use of a training set extracted from this dataset, we were able to create a KNN classification model and test our testing set to determine our accuracy. We then ran our unknown bank notes through our model and created visualizations to understand our bank notes better. Our model classified the unknown bank notes with 100% accuracy.

## Introduction

Introduction paragraph goes in here.

## Methods & Results

### Import All Necessary Libraries

In [1]:
# Import all the libraries we need for the analysis
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

### Loading Data From the Web

In [2]:
# Loading data from the web
# Data was downloaded from https://www.kaggle.com/datasets/ritesaluja/bank-note-authentication-uci-data?resource=download
bill_url = "https://raw.githubusercontent.com/DSCI-310-2025/dsci-310-group-python2/refs/heads/main/data/BankNote_Authentication.csv"
bill_data = pd.read_csv(bill_url)

### Wrangle and Clean Data

In [3]:
# See if there are missing values
missing_values = bill_data.isna().sum().sum()
print(missing_values)

0


### Summary of Dataset

In [4]:
# Create count table to see how many of each class there are
count_table = bill_data.groupby('class').size().reset_index(name='Count')
count_table['Percentage'] = 100 * count_table['Count'] / len(bill_data)

count_table

,class,Count,Percentage
0,0,762,55.539359
1,1,610,44.460641


#### Table 1. Authentic (0) to Fake (1) Proportion in Dataset

In [5]:
# Split data into training and testing sets
bill_train, bill_test = train_test_split(bill_data, test_size=0.25, stratify=bill_data['class'], random_state=1)
# Take a look at first 5 rows. NOT OFFICIALLY A TABLE
bill_train.head(5)

,variance,skewness,curtosis,entropy,class
892,0.21431,-0.69529,0.87711,0.29653,1
465,-2.69890,12.19840,0.67661,-8.54820,0
57,5.80700,5.00970,-2.23840,0.43878,0
1166,-2.19790,-2.12520,1.71510,0.45171,1
223,4.64640,10.53260,-4.58520,-4.20600,0


In [6]:
# Create class proportions table
class_proportions_table = bill_train.groupby('class').size().reset_index(name='Count')
class_proportions_table['Percentage'] = 100 * class_proportions_table['Count'] / len(bill_train)

class_proportions_table

# We notice that our "class" proportions were preserved when we split the data

,class,Count,Percentage
0,0,572,55.587949
1,1,457,44.412051


#### Table 2. Authentic (0) to Fake (1) Proportion in Trainin Dataset

In [7]:
# Compute mean for each factor
means_per_column = bill_train.iloc[:, :-1].mean().reset_index()
means_per_column.columns = ['Factor', 'Mean_Value']

means_per_column

,Factor,Mean_Value
0,variance,0.398695
1,skewness,1.837843
2,curtosis,1.462280
3,entropy,-1.192189


#### Table 2. Mean of Factors

In [8]:
# Compute max for each factor
max_per_column = bill_train.iloc[:, :-1].max().reset_index()
max_per_column.columns = ['Factor', 'Maximum_Value']

max_per_column

,Factor,Maximum_Value
0,variance,6.8248
1,skewness,12.6247
2,curtosis,17.6772
3,entropy,2.1625


#### Table 3. Maximium of Factors

In [9]:
# Compute min for each factor
min_per_column = bill_train.iloc[:, :-1].min().reset_index()
min_per_column.columns = ['Factor', 'Minimum_Value']

min_per_column

,Factor,Minimum_Value
0,variance,-7.0421
1,skewness,-13.6779
2,curtosis,-5.2861
3,entropy,-8.5482


#### Table 4. Minimum of Factors

## Visualizations of the Dataset

### Figure 1. Distribution of Variance Grouped By Authentic or Fake

In [10]:
# Plot histogram of variance column distribution

plt.figure(figsize=(8, 6))
sns.histplot(data=bill_train, x='variance', hue='class', element='bars', bins=30, kde=True)
plt.xlabel("Variance")
plt.ylabel("Value")
plt.title("Distribution of Variance Labeled as 0 or 1")
plt.legend(title="Class", labels=["1 (Fake)", "0 (Authentic)"])
plt.show()

### Figure 2. Distribution of Skewness Grouped By Authentic or Fake

In [11]:
# Plot histogram of skewness column distribution

plt.figure(figsize=(8, 6))
sns.histplot(data=bill_train, x='skewness', hue='class', element='bars', bins=30, kde=True)
plt.xlabel("Skewness")
plt.ylabel("Value")
plt.title("Distribution of Skewness Labeled as 0 or 1")
plt.legend(title="Class", labels=["1 (Fake)", "0 (Authentic)"])
plt.show()

### Figure 3. Distribution of Curtosis Grouped By Authentic or Fake

In [12]:
# Plot histogram of curtosis column distribution

plt.figure(figsize=(8, 6))
sns.histplot(data=bill_train, x='curtosis', hue='class', element='bars', bins=30, kde=True)
plt.xlabel("Curtosis")
plt.ylabel("Value")
plt.title("Distribution of Curtosis Labeled as 0 or 1")
plt.legend(title="Class", labels=["1 (Fake)", "0 (Authentic)"])
plt.show()

### Figure 4. Distribution of Entropy Grouped By Authentic or Fake

In [13]:
# Plot histogram of entropy column distribution

plt.figure(figsize=(8, 6))
sns.histplot(data=bill_train, x='entropy', hue='class', element='bars', bins=30, kde=True)
plt.xlabel("Entropy")
plt.ylabel("Value")
plt.title("Distribution of Entropy Labeled as 0 or 1")
plt.legend(title="Class", labels=["1 (Fake)", "0 (Authentic)"])
plt.show()

#### Summary of Visualizations

The entropy histograms (Figure 4) have very similar distributions for authentic and fake bills as they both have their modes around 0. Since the distributions are very similar, it is very unlikely that entropy is a driving factor is determining authentic or fake bills. The same can be said for the curtosis histogram (Figure 3) as authentic and fake bills have similar distributions with their modes around 0 as well. Curtosis is also unlikely to be a driving force. The variance distributions (Figure 1) are very different. Both authentic and fake bills have a bell curve distribution with the fake bills having a mode around -2.5 and the authentic bills having a mode around 4. This suggests that variance is a very strong driving force in determining whether a bill is authentic or fake. The skewness histograms (Figure 2) have somewhat different distributions. The fake bills have their mode at around 2.5 while the authentic bills have their mode at around 8. The distributions overlap quite a bit suggesting that skewness is not as big of a driving factor as variance. These histograms will allow us to understand what we should expect with our unknown bill once the bill is identified. We can use these histograms to see if our expectations line up with the unknown bill.

## Data Analysis

### Cross validation

We tried standard scaler but got better results without it.

In [14]:
X_train = bill_train.drop('class', axis=1)  # features
y_train = bill_train['class']              # target

neighbors = range(1, 26)  # testing 1 to 25 neighbors
cv_scores = []

for k in neighbors:
    # Update the pipeline parameter for number of neighbors
    
    KNN = KNeighborsClassifier(n_neighbors= k)
    
    # Perform 5-fold cross-validation and record the mean accuracy
    scores = cross_val_score(KNN, X_train, y_train, cv=2)
    cv_scores.append(scores.mean())

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(neighbors, cv_scores, marker='o')
plt.xlabel('Number of Neighbors')
plt.ylabel('Mean CV Accuracy')
plt.title('KNN Accuracy vs. Number of Neighbors')
plt.grid(True)
plt.show()

### Parameter optimization

Seems like even with one neighbour we are getting the best result, so let's go ahead and use n_neighbours = 1 in our pipeline

### Applying our findings on test data

In [15]:
X_test = bill_test.drop('class', axis=1)  # features
y_test = bill_test['class']              # target

FinalKNN = KNeighborsClassifier(n_neighbors= 1)

FinalKNN.fit(X_train,y_train)

FinalKNN.score(X_test,y_test)


1.0

### Predict Test Data Results
As we can see, we achieve score of 1. Which means we with no error we are predicting everything right. Lets explore that a bit more in terms of numbers of correct predictions. 

### Confusion Matrix

In [16]:
# Make predictions on the test set
y_pred = FinalKNN.predict(X_test)

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Optionally, print a classification report for additional metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[190   0]
 [  0 153]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       190
           1       1.00      1.00      1.00       153

    accuracy                           1.00       343
   macro avg       1.00      1.00      1.00       343
weighted avg       1.00      1.00      1.00       343



### Interpretation:*  
Every test sample was classified correctly. There are no misclassifications, which implies that the model perfectly distinguishes between the two classes. 
### Key Metrics

- **Precision:**  
  - Class 0: 1.00  
  - Class 1: 1.00  
  Indicates that every instance predicted as a specific class was correct.

- **Recall:**  
  - Class 0: 1.00  
  - Class 1: 1.00  
  Indicates that the model successfully identified all instances for each class.

- **F1-Score:**  
  - Class 0: 1.00  
  - Class 1: 1.00  
  The harmonic mean of precision and recall, confirming overall perfect performance.

- **Support:**  
  - Class 0: 190 instances  
  - Class 1: 153 instances  
  The number of actual instances in each class within the test set.

- **Accuracy:**  
  - Overall: 1.00 (100%)  
  All 343 test samples were classified correctly.

The classifier achieved **100% accuracy** on the test set, with perfect precision, recall, and F1-scores for both classes. 


## Discussion

The classifier perfectly predicted all 190 real banknotes and 153 fake banknotes in the test set. This is not what we expected to find. While we did expect our model to do well since our 4 input variables likely play a major role in detecting forged banknotes, we did not expect it to perfectly predict every banknote in the test set.

This result suggests that our current model is identifying all the patterns in the dataset and is able to predict whether or not a banknote is forged or not with 100% accuracy. However, this model still needs more testing. We could be overfitting on our available data. Our test set may also not capture all the patterns of a banknote the model might encounter, as it only has 343 samples. This test could have failed to include some rare or outlier patterns that the model may encounter if it were tested on a different dataset. It may even be the case that our entire dataset may not be representative of the real world and this could potentially cause our model to perform poorly on unseen data. 

This analysis raises several potential future questions that could evaluate how this model would fare in real world scenarios:
- Would our model still be generalizable for larger or more complex datasets or would it lead to lower accuracy?
- Which of the 4 features between variance, skewness, kurtosis, and entropy are the most important for authenticating banknotes and by how much?
- Is there covariance between the 4 features? Does this covariance make it easier to detect fake banknotes?
- What are the patterns that make a counterfeit banknote stand out from a genuine banknote? Could a forged banknote be designed to avoid these patterns and not be detected by the classification model?

### References

references go here